# Example Multiprocessing Simple: Process x


<i>process_target_x</i> is the target function for process <b>x</b>. It is in a file called 
<br>
<b>example_multiprocessing_simple_process_x.py</b>.


The code for the target function for process <b>y</b> is in a file called 
<br>
<b>example_multiprocessing_simple_process_y.py</b>

The parameter for <i>process_target_x</i> is <b>dict_queues</b> which is a dict of
multiprocessing queues. We use a dict rather than passing queues directly. In this
example the input queues for processes <b>x</b> and <b>y</b> are
<b>dict_queues['x']</b> and <b>dict_queues['y']</b>, respectively.

The steps of this target function are as follows:
<ol>
    <li>
        <b><i>Specify the input queue, <i>Stream.scheduler.input_queue</i>,
        for this process</i></b>. 
        The input queue is obtained from the parameter <i>dict_queues.</i>
        <br>
        <code>
            Stream.scheduler.input_queue = dict_queues['x']
        </code>
    </li>
    <li>
        <b><i>Create the streams in this process.</i></b>.
        <br>
        <code>
            x = Stream(name='x')
        </code>
    </li>
    <li>
        <b><i>Create external streams in this process.</i></b>.
        An external stream is specified by two parameters:
        (a) the name of the stream in the receiving process, and (b) the input queue of
        the receiving process.
        <br>
        <code>
            y = ExternalStream(name='y', queue=dict_queues['y'])
        </code>
    </li>
    <li>
        <b><i>Declare the callback functions used in this process</i></b>. The callback function
        prints the new item of the stream and then appends the incremented item to 
        the external stream <b>y</b>.
        <code>
            print('message received by process x: ', stream_item)
            y.append(stream_item+1)
        </code>
    </li>
    <li>
        <b><i>Create agents in this process</i></b>
        <code>
            single_item(in_stream=x, func=callback_x)
        </code>
    </li>
    <li>
        <b><i>Start this process by starting its scheduler</i></b>:
        <code>
            Stream.scheduler.start()
        </code>
    </li>
</ol>

In [1]:
from stream import Stream, ExternalStream
from example_operators import single_item
    
def process_target_x(dict_queues):
    
    #-------------------------------------------
    # 1. SPECIFY INPUT QUEUES FOR THE PROCESSES
    #-------------------------------------------
    # Specify that the input stream for THIS process
    # is dict_queues['x']
    Stream.scheduler.input_queue = dict_queues['x']
    
    #-------------------------------------------
    # 2. SPECIFY STREAMS IN THIS PROCESS
    #-------------------------------------------
    x = Stream(name='x')
    
    #-------------------------------------------
    # 3. SPECIFY EXTERNAL STREAMS 
    #-------------------------------------------
    y = ExternalStream(name='y', queue=dict_queues['y'])
    
    #-------------------------------------------
    # 4. SPECIFY CALLBACK FUNCTIONS IN THIS PROCESS
    #-------------------------------------------
    def callback_x(stream_item):
        print('message received by process x: ', stream_item)
        y.append(stream_item+1)
        
    #-------------------------------------------
    # 5. SPECIFY AGENTS IN THIS PROCESS
    #-------------------------------------------
    single_item(in_stream=x, func=callback_x)
    
    #-------------------------------------------
    # 6. START SCHEDULER AND THUS START THIS PROCESS
    #-------------------------------------------
    Stream.scheduler.start()